This example is to construct a student-$t$ process regression model.

Import some packages required to this example.

In [ ]:
%matplotlib inline

import numpy as np

from bayeso import covariance
from bayeso.tp import tp
from bayeso.gp import gp
from bayeso.utils import utils_covariance
from bayeso.utils import utils_plotting

Define parameters we need.

In [ ]:
use_tex = False
num_test = 200
str_cov = 'matern52'

Define training and test data points, and their evaluations.

In [ ]:
X_train = np.array([
    [-3.0],
    [-2.0],
    [-1.0],
    [1.1],
    [1.2],
    [2.5],
])
Y_train = np.cos(X_train) + 10.0 + np.random.randn(*X_train.shape)

X_test = np.linspace(-3, 3, num_test)
X_test = X_test.reshape((num_test, 1))
Y_test = np.cos(X_test) + 10.0

Sample `num_samples` functions from a prior distribution and plot them.

In [ ]:
mu = np.zeros(num_test)
hyps = utils_covariance.get_hyps(str_cov, 1, use_gp=False)
Sigma = covariance.cov_main(str_cov, X_test, X_test, hyps, True)

Ys = tp.sample_functions(hyps['dof'], mu, Sigma, num_samples=200)
utils_plotting.plot_gp_via_sample(X_test, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

hyps = utils_covariance.get_hyps(str_cov, 1)
Sigma = covariance.cov_main(str_cov, X_test, X_test, hyps, True)

Ys = gp.sample_functions(mu, Sigma, num_samples=200)
utils_plotting.plot_gp_via_sample(X_test, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

Predict a posterior predictive distribution with fixed hyperparameters and plot it.
Also, sample `num_samples` functions from the predictive posterior distribution.

In [ ]:
hyps = utils_covariance.get_hyps(str_cov, 1, use_gp=False)
nu, mu, sigma, Sigma = tp.predict_with_hyps(X_train, Y_train, X_test, hyps, str_cov=str_cov)
utils_plotting.plot_gp_via_distribution(
    X_train, Y_train, X_test, mu, sigma,
    Y_test=Y_test, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = tp.sample_functions(nu, mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

hyps = utils_covariance.get_hyps(str_cov, 1)
mu, sigma, Sigma = gp.predict_with_hyps(X_train, Y_train, X_test, hyps, str_cov=str_cov)
utils_plotting.plot_gp_via_distribution(
    X_train, Y_train, X_test, mu, sigma,
    Y_test=Y_test, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = gp.sample_functions(mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

Predict a posterior predictive distribution with learned hyperparameters and plot it.
Also, sample `num_samples` functions from the posterior predictive distribution.

In [ ]:
nu, mu, sigma, Sigma = tp.predict_with_optimized_hyps(X_train, Y_train, X_test, str_cov=str_cov, fix_noise=True,
                                                      debug=True)
utils_plotting.plot_gp_via_distribution(
    X_train, Y_train, X_test, mu, sigma,
    Y_test=Y_test, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = tp.sample_functions(nu, mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

mu, sigma, Sigma = gp.predict_with_optimized_hyps(X_train, Y_train, X_test, str_cov=str_cov, fix_noise=True,
                                                  debug=True)
utils_plotting.plot_gp_via_distribution(
    X_train, Y_train, X_test, mu, sigma,
    Y_test=Y_test, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = gp.sample_functions(mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

Define prior functions.
`cosine` is a function with a cosine function,
`linear_down` is a function with a decreasing function,
and `linear_up` is a function with a increasing function.

In [ ]:
def cosine(X):
    return np.cos(X)

def linear_down(X):
    list_up = []
    for elem_X in X:
        list_up.append([-0.5 * np.sum(elem_X)])
    return np.array(list_up)

def linear_up(X):
    list_up = []
    for elem_X in X:
        list_up.append([0.5 * np.sum(elem_X)])
    return np.array(list_up)

Define training and test data points, and their evaluations.

In [ ]:
X_train_ = np.array([
    [-3.0],
    [-2.0],
    [-1.0],
])
Y_train_ = np.cos(X_train_) + 2.0 + np.random.randn(*X_train_.shape)
X_test_ = np.linspace(-3, 6, num_test)
X_test_ = X_test_.reshape((num_test, 1))
Y_test_ = np.cos(X_test_) + 2.0

Predict a posterior predictive distribution with a `cosine` function, and plot it.
In addition, sample `num_samples` functions from the distribution, and plot them.

In [ ]:
prior_mu = cosine
nu, mu, sigma, Sigma = tp.predict_with_optimized_hyps(X_train_, Y_train_, X_test_,
                                                      str_cov=str_cov, prior_mu=prior_mu,
                                                      debug=True)
utils_plotting.plot_gp_via_distribution(
    X_train_, Y_train_, X_test_, mu, sigma,
    Y_test=Y_test_, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = tp.sample_functions(nu, mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test_, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

Predict a posterior predictive distribution with a `linear_down` function, and plot it.
In addition, sample `num_samples` functions from the distribution, and plot them.

In [ ]:
prior_mu = linear_down
nu, mu, sigma, Sigma = tp.predict_with_optimized_hyps(X_train_, Y_train_, X_test_,
                                                      str_cov=str_cov, prior_mu=prior_mu,
                                                      debug=True)
utils_plotting.plot_gp_via_distribution(
    X_train_, Y_train_, X_test_, mu, sigma,
    Y_test=Y_test_, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = tp.sample_functions(nu, mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test_, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')

Predict a posterior predictive distribution with a `linear_up` function, and plot it.
In addition, sample `num_samples` functions from the distribution, and plot them.

In [ ]:
prior_mu = linear_up
nu, mu, sigma, Sigma = tp.predict_with_optimized_hyps(X_train_, Y_train_, X_test_,
                                                      str_cov=str_cov, prior_mu=prior_mu,
                                                      debug=True)
utils_plotting.plot_gp_via_distribution(
    X_train_, Y_train_, X_test_, mu, sigma,
    Y_test=Y_test_, use_tex=use_tex,
    str_x_axis='$x$', str_y_axis='$y$'
)

Ys = tp.sample_functions(nu, mu.flatten(), Sigma, num_samples=5)
utils_plotting.plot_gp_via_sample(X_test_, Ys, use_tex=use_tex,
                                  str_x_axis='$x$', str_y_axis='$y$')